# Visualization of feature data

This notebook produces a selection of feature maps. 

**Note: requires certificate/access to the ViEWS database.**

In [1]:
# Basics
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cbook as cbook
# sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.ensemble import AdaBoostRegressor
from sklearn import linear_model
from sklearn.metrics import mean_squared_error

# xgboost
from xgboost import XGBRegressor
from xgboost import XGBClassifier
from xgboost import XGBRFClassifier
from xgboost import XGBRFRegressor

from lightgbm import LGBMClassifier
from lightgbm import LGBMRegressor

# Views 3
from viewser.operations import fetch
from viewser import Queryset, Column
import views_runs
from views_partitioning import data_partitioner, legacy
from stepshift import views
import views_dataviz

from views_runs import Storage, StepshiftedModels
from views_partitioning.data_partitioner import DataPartitioner
from views_runs import run_result

# Mapper
import geopandas as gpd

from views_dataviz.map import mapper, utils
from views_dataviz import color
from views_dataviz.map.presets import ViewsMap

from views_forecasts.extensions import *

# from views_transformation_library import utilities

import sqlalchemy as sa
from ingester3.config import source_db_path

# Other packages

from datetime import datetime

import seaborn as sns

#import pygam

import scipy
from scipy import signal

# CM feature maps
## GED, libdem, gdp per capita

In [2]:
qs = (Queryset("features_cm", "country_month")


    # target variable
    .with_column(Column("ln_ged_sb", from_table = "ged2_cm", from_column = "ged_sb_best_sum_nokgi")
        .transform.ops.ln()
        )
      .with_column(Column("ln_ged_ns", from_table = "ged2_cm", from_column = "ged_ns_best_sum_nokgi")
        .transform.ops.ln()
        )
      .with_column(Column("ln_ged_os", from_table = "ged2_cm", from_column = "ged_os_best_sum_nokgi")
        .transform.ops.ln()
        )
           
      
     )

In [3]:
features_cm=qs.publish().fetch()

 .      o   

In [4]:
features_cm['gdp_pc']=features_cm['wdi_ny_gdp_mktp_kd']/features_cm['wdi_sp_pop_totl']

KeyError: 'wdi_ny_gdp_mktp_kd'

In [7]:
features_cm['ln_gdp_pc']=np.log(features_cm['gdp_pc'])

KeyError: 'gdp_pc'

In [4]:
features_cm

ln_ged_sb  ln_ged_ns  ln_ged_os
month_id country_id                                 
1        1                 NaN        NaN        NaN
         2                 NaN        NaN        NaN
         3                 NaN        NaN        NaN
         4                 NaN        NaN        NaN
         5                 NaN        NaN        NaN
...                        ...        ...        ...
852      242               NaN        NaN        NaN
         243               NaN        NaN        NaN
         244               NaN        NaN        NaN
         245               NaN        NaN        NaN
         246               NaN        NaN        NaN

[158230 rows x 3 columns]

In [5]:
engine = sa.create_engine(source_db_path)
gdf_c = gpd.GeoDataFrame.from_postgis(
    "SELECT id as country_id, in_africa, in_me, geom FROM prod.country", 
    engine, 
    geom_col='geom'
)
gdf_c = gdf_c.to_crs(4326)

features_cm_gdf = features_cm.join(gdf_c.set_index("country_id"))
features_cm_gdf = gpd.GeoDataFrame(features_cm_gdf, geometry="geom")

In [6]:
savepath='/Users/angli742/Desktop/'

### GED maps, double log

In [7]:
conf_types=['sb','ns','os']
tickvalues=np.array([0,3,10,30,100,300])
ticklabels=[str(tv) for tv in tickvalues]

tickvalues=np.log(np.log(tickvalues+1)+1)

t0s=range(524,525)
bbox="africa_middle_east"
cmap='viridis'#'rainbow'
for t0 in t0s:
    for type in conf_types:
        column='ln_ged_'+type
        features_cm_gdf['log_'+column]=np.log(features_cm_gdf[column]+1) 
        data_actual=features_cm_gdf[['log_'+column,'geom']].loc[t0]
        column='log_'+column
        titlestring=''
        plot = ViewsMap(
            width=10,
            label=f"GED fatalities {type}",
            title="",
            scale=None,
            bbox=bbox
        ).add_layer(
            data_actual,
            edgecolor="black",
            linewidth=0.2,
            column=column,
        inform_colorbar=True,
        cmap=cmap,
        vmin=tickvalues[0],vmax=tickvalues[-1]
    )

        ax=plot.ax
        fg=gdf_c.plot(ax=ax,edgecolor='black',linewidth=0.2,facecolor='None')
       # fg=gdf_c.plot(ax=ax,edgecolor='gray',linewidth=1.0,facecolor='None')
        figure=plot.fig
        fontdict={'fontsize':20}
        fig=plot.fig

        plot.cbar.set_ticks(tickvalues)
        plot.cbar.set_ticklabels(ticklabels)
        plot.cbar.set_label('GED fatalities')

        plot.save(savepath+'ame_cm_fatalities_'+type+'_'+cmap+'_'+str(t0)+'.png')

### GED maps, single log

In [9]:
conf_types=['sb','ns','os']
tickvalues=np.array([0,3,10,30,100,300])
ticklabels=[str(tv) for tv in tickvalues]

tickvalues=np.log(tickvalues+1)

        
t0s=range(524,525)
bbox="africa_middle_east"
cmap='viridis_r'#'rainbow'
for t0 in t0s:
    for type in conf_types:
        column='ln_ged_'+type
        data_actual=features_cm_gdf[[column,'geom']].loc[t0]
        titlestring=''
        plot = ViewsMap(
            width=10,
            label=f"GED fatalities {type}",
            title="",
            scale=None,
            bbox=bbox
        ).add_layer(
            data_actual,
            edgecolor="black",
            linewidth=0.2,
            column=column,
        inform_colorbar=True,
        cmap=cmap,
        vmin=tickvalues[0],vmax=tickvalues[-1]
    )

        ax=plot.ax
        fg=gdf_c.plot(ax=ax,edgecolor='black',linewidth=0.2,facecolor='None')
       # fg=gdf_c.plot(ax=ax,edgecolor='gray',linewidth=1.0,facecolor='None')
        figure=plot.fig
        fontdict={'fontsize':20}
        fig=plot.fig

        plot.cbar.set_ticks(tickvalues)
        plot.cbar.set_ticklabels(ticklabels)
        plot.cbar.set_label('GED fatalities')

        plot.save(savepath+'ame_cm_fatalities_singlelog'+type+'_'+cmap+'_'+str(t0)+'.png')

### Liberal democracy

In [15]:
column="libdem"
tickvalues=np.array([0,0.2,0.4,0.6,0.8,1.0])
ticklabels=[str(tv) for tv in tickvalues]
#tickvalues=np.log(tickvalues+1)

t0s=range(492,493)
bbox="africa_middle_east"
for t0 in t0s:
    data_actual=features_cm_gdf[[column,'geom']].loc[t0]
    titlestring=''
    plot = ViewsMap(
        width=10,
        label=f"VDEM liberal democracy",
        title="",
        scale=None,
        bbox=bbox
    ).add_layer(
        data_actual,
        edgecolor="black",
        linewidth=0.2,
        column=column,
        inform_colorbar=True,
        cmap='viridis_r', # change color scale here
        vmin=tickvalues[0],vmax=tickvalues[-1]
    )

    ax=plot.ax
    fg=gdf_c.plot(ax=ax,edgecolor='black',linewidth=0.2,facecolor='None')
    #fg=gdf_c.plot(ax=ax,edgecolor='gray',linewidth=0.5,facecolor='None')
    figure=plot.fig
    fontdict={'fontsize':20}
    fig=plot.fig



    plot.cbar.set_ticks(tickvalues)
    plot.cbar.set_ticklabels(ticklabels)
    plot.cbar.set_label('VDEM liberal democracy score')

    plot.save(savepath+'ame_cm_libdem'+'_'+cmap+'_'+str(t0)+'.png')

## GDP per capita

In [16]:
column="ln_gdp_pc"
tickvalues=np.array([150,500,1500,5000,15000,50000])
ticklabels=[str(tv) for tv in tickvalues]
tickvalues=np.log(tickvalues+1)
t0s=range(506,507)
bbox="africa_middle_east"
for t0 in t0s:
    data_actual=features_cm_gdf[[column,'geom']].loc[t0]
    titlestring=''
    plot = ViewsMap(
        width=10,
        label=f"GDP per capita",
        title="",
        scale=None,
        bbox=bbox
    ).add_layer(
        data_actual,
        edgecolor="black",
        linewidth=0.2,
        column=column,
        inform_colorbar=True,
        cmap='viridis_r',
        vmin=tickvalues[0],vmax=tickvalues[-1]
    )

#    ax=plot.ax
#    fg=gdf_c.plot(ax=ax,edgecolor='gray',linewidth=1.0,facecolor='None')
#    figure=plot.fig
#    fontdict={'fontsize':20}
#    fig=plot.fig



    plot.cbar.set_ticks(tickvalues)
    plot.cbar.set_ticklabels(ticklabels)
    plot.cbar.set_label('GDP per capita (constant USD, market prices)')

    plot.save(savepath+'ame_cm_gdp_pc'+'_'+cmap+'_'+str(t0)+'.png')

## Infant mortality rate


In [17]:
column="wdi_sp_dyn_imrt_in"
tickvalues=np.array([0,20,40,60,80,100])
ticklabels=[str(tv) for tv in tickvalues]
#tickvalues=np.log(tickvalues+1)
t0s=range(506,507)
bbox="africa_middle_east"
for t0 in t0s:
    data_actual=features_cm_gdf[[column,'geom']].loc[t0]
    titlestring=''
    plot = ViewsMap(
        width=10,
        label=f"Infant mortality",
        title="",
        scale=None,
        bbox=bbox
    ).add_layer(
        data_actual,
        edgecolor="black",
        linewidth=0.2,
        column=column,
        inform_colorbar=True,
        cmap='viridis',
        vmin=tickvalues[0],vmax=tickvalues[-1]
    )

#    ax=plot.ax
#    fg=gdf_c.plot(ax=ax,edgecolor='gray',linewidth=1.0,facecolor='None')
#    figure=plot.fig
#    fontdict={'fontsize':20}
#    fig=plot.fig



    plot.cbar.set_ticks(tickvalues)
    plot.cbar.set_ticklabels(ticklabels)
    plot.cbar.set_label('Infant mortaility rate (per 1 000 live births)')

    plot.save(savepath+'ame_cm_imr'+'_'+cmap+'_'+str(t0)+'.png')

# PGM feature maps
## GED data

In [11]:
qs = (Queryset("FCDO_pgm", "priogrid_month")


    # target variable
    .with_column(Column("ln_ged_sb", from_table = "ged2_pgm", from_column = "ged_sb_best_sum_nokgi")
        .transform.ops.ln()     
        )
     .with_column(Column("ln_ged_ns", from_table = "ged2_pgm", from_column = "ged_ns_best_sum_nokgi")
        .transform.ops.ln()     
        )
     .with_column(Column("ln_ged_os", from_table = "ged2_pgm", from_column = "ged_os_best_sum_nokgi")
        .transform.ops.ln()     
        )
     )


fcdo_pgm=qs.publish().fetch()


 .      o      O      O      o       .      o   

In [12]:
savepath='/Users/angli742/Desktop/'

In [13]:
engine = sa.create_engine(source_db_path)
gdf = gpd.GeoDataFrame.from_postgis(
    "SELECT id as pg_id, in_africa, in_me, geom FROM prod.priogrid", 
    engine, 
    geom_col='geom'
)
gdf = gdf.to_crs(4326)
gdf['priogrid_gid']=gdf['pg_id']
#fcdo_pgm_gdf = fcdo_pgm.join(gdf.set_index("priogrid_id"))
fcdo_pgm_gdf = fcdo_pgm.join(gdf.set_index("priogrid_gid")) # temporary, something's off in viewser
#fcdo_pgm_gdf = fcdo_pgm.join(gdf.set_index("pg_id"))
fcdo_pgm_gdf = gpd.GeoDataFrame(fcdo_pgm_gdf, geometry="geom")


#fcdo_cm_gdf = fcdo_cm.join(gdf_c.set_index("country_id"))
#fcdo_cm_gdf = gpd.GeoDataFrame(fcdo_cm_gdf, geometry="geom")


### Single log

In [14]:
conf_types=['sb','ns','os']
tickvalues=np.array([0,3,10,30,100,300])
ticklabels=[str(tv) for tv in tickvalues]
tickvalues=np.log(tickvalues+1)

t0s=range(524,525)
bbox="africa_middle_east"
cmap='viridis'   #'rainbow'
for t0 in t0s:
    for type in conf_types:
        column='ln_ged_'+type
        data_actual=fcdo_pgm_gdf[[column,'geom']].loc[t0]
        titlestring=''
        plot = ViewsMap(
            width=10,
            label=f"GED fatalities {type}",
            title="",
            scale=None,
            bbox=bbox
        ).add_layer(
            data_actual,
            edgecolor="black",
            linewidth=0.2,
            column=column,
        inform_colorbar=True,
        cmap=cmap,
        vmin=tickvalues[0],vmax=tickvalues[-1]
    )

        ax=plot.ax
        fg=gdf_c.plot(ax=ax,edgecolor='gray',linewidth=0.8,facecolor='None')
        figure=plot.fig
        fontdict={'fontsize':20}
        fig=plot.fig



        plot.cbar.set_ticks(tickvalues)
        plot.cbar.set_ticklabels(ticklabels)
        plot.cbar.set_label('GED fatalities')

        plot.save(savepath+'ame_pgm_fatalities_'+type+'_'+cmap+'_'+str(t0)+'.png')

### Double log

In [15]:
conf_types=['sb','ns','os']
tickvalues=np.array([0,3,10,30,100,300])
ticklabels=[str(tv) for tv in tickvalues]
tickvalues=np.log(np.log(tickvalues+1)+1)

t0s=range(542,525)
bbox="africa_middle_east"
cmap='viridis'#'rainbow'

for t0 in t0s:
    for type in conf_types:
        column='ln_ged_'+type
        fcdo_pgm_gdf['log_'+column]=np.log(fcdo_pgm_gdf[column]+1) 
        data_actual=fcdo_pgm_gdf[['log_'+column,'geom']].loc[t0]
        column='log_'+column
        titlestring=''
        plot = ViewsMap(
            width=10,
            label=f"GED fatalities {type}",
            title="",
            scale=None,
            bbox=bbox
        ).add_layer(
            data_actual,
            edgecolor="black",
            linewidth=0.2,
            column=column,
        inform_colorbar=True,
        cmap=cmap,
        vmin=tickvalues[0],vmax=tickvalues[-1]
    )

        ax=plot.ax
        fg=gdf_c.plot(ax=ax,edgecolor='black',linewidth=0.2,facecolor='None')
       # fg=gdf_c.plot(ax=ax,edgecolor='gray',linewidth=1.0,facecolor='None')
        figure=plot.fig
        fontdict={'fontsize':20}
        fig=plot.fig

        plot.cbar.set_ticks(tickvalues)
        plot.cbar.set_ticklabels(ticklabels)
        plot.cbar.set_label('GED fatalities')

        plot.save(savepath+'ame_pgm_fatalities_'+type+'_'+cmap+'_'+str(t0)+'.png')

# Changes to GED actuals between two months

## CM changes

In [30]:
conf_types=['sb','ns','os']
delta=3 # 1 month
for conf_type in conf_types:
    column='ln_ged_'+conf_type
    actual=np.exp(features_cm_gdf[column])-1
    
    shifted=np.exp(features_cm_gdf[column].groupby(level=1).shift(periods=delta))-1    
    
    features_cm_gdf['ln_delta_'+str(delta)+'_'+column]=np.sign(actual-shifted)*np.log(np.abs(actual-shifted)+1)


tickvalues=np.array([-300,-30,-3,3,30,300])
ticklabels=[str(tv) for tv in tickvalues]

tickvalues=np.sign(tickvalues)*np.log(np.abs(tickvalues)+1)

t0s=range(512,513) # From start of month A, to start of (but not including) month B
bbox="africa_middle_east"
cmap='bwr'#'rainbow'
for t0 in t0s:
    for type in conf_types:
        column='ln_delta_'+str(delta)+'_ln_ged_'+type
    
        data_actual=features_cm_gdf[[column,'geom']].loc[t0]

        titlestring=''
        plot = ViewsMap(
            width=10,
            label=f"$\Delta$(GED fatalities) {type}",
            title="",
            scale=None,
            bbox=bbox
        ).add_layer(
            data_actual,
            edgecolor="black",
            linewidth=0.2,
            column=column,
        inform_colorbar=True,
        cmap=cmap,
        vmin=tickvalues[0],vmax=tickvalues[-1]
    )

        ax=plot.ax
        fg=gdf_c.plot(ax=ax,edgecolor='black',linewidth=0.2,facecolor='None')
       # fg=gdf_c.plot(ax=ax,edgecolor='gray',linewidth=1.0,facecolor='None')
        figure=plot.fig
        fontdict={'fontsize':20}
        fig=plot.fig

        plot.cbar.set_ticks(tickvalues)
        plot.cbar.set_ticklabels(ticklabels)
        if abs(delta)==1:
            mnth='month'
        else:
            mnth='months'
        plot.cbar.set_label('$\Delta$(GED fatalities) over '+str(delta)+' '+mnth)

        plot.save(savepath+'ame_cm_delta_fatalities_over_'+str(delta)+'_'+type+'_'+cmap+'_'+str(t0)+'.png')

## PGM changes


In [31]:
conf_types=['sb','ns','os']
delta=1 # 1 month
for conf_type in conf_types:
    column='ln_ged_'+conf_type
    actual=np.exp(fcdo_pgm_gdf[column])-1
    
    shifted=np.exp(fcdo_pgm_gdf[column].groupby(level=1).shift(periods=delta))-1
    
    fcdo_pgm_gdf['ln_delta_'+str(delta)+'_'+column]=np.sign(actual-shifted)*np.log(np.abs(actual-shifted)+1)


tickvalues=np.array([-300,-30,-3,3,30,300])
ticklabels=[str(tv) for tv in tickvalues]

tickvalues=np.sign(tickvalues)*np.log(np.abs(tickvalues)+1)

t0s=range(512,513)
bbox="africa_middle_east"
cmap='bwr'#'rainbow'
for t0 in t0s:
    for type in conf_types:
        column='ln_delta_'+str(delta)+'_ln_ged_'+type
    
        data_actual=fcdo_pgm_gdf[[column,'geom']].loc[t0]

        titlestring=''
        plot = ViewsMap(
            width=10,
            label=f"$\Delta$(GED fatalities) {type}",
            title="",
            scale=None,
            bbox=bbox
        ).add_layer(
            data_actual,
            edgecolor="black",
            linewidth=0.0, # grid lines
            column=column,
        inform_colorbar=True,
        cmap=cmap,
        vmin=tickvalues[0],vmax=tickvalues[-1]
    )

        ax=plot.ax
        fg=gdf_c.plot(ax=ax,edgecolor='black',linewidth=0.2,facecolor='None') # cm lines
       # fg=gdf_c.plot(ax=ax,edgecolor='gray',linewidth=1.0,facecolor='None')
        figure=plot.fig
        fontdict={'fontsize':20}
        fig=plot.fig

        plot.cbar.set_ticks(tickvalues)
        plot.cbar.set_ticklabels(ticklabels)
        if abs(delta)==1:
            mnth='month'
        else:
            mnth='months'
        plot.cbar.set_label('$\Delta$(GED fatalities) over '+str(delta)+' '+mnth)

        plot.save(savepath+'ame_pgm_delta_fatalities_over_'+str(delta)+'_'+type+'_'+cmap+'_'+str(t0)+'.png')